In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import cv2
import matplotlib.pyplot as plt

In [2]:
import os
d = os.listdir()
path = []
path.append(os.listdir('.\\Kaggle\\yes'))
path.append(os.listdir('.\\Kaggle\\no'))
path = {'yes':path[0],'no':path[1]}

In [4]:
x,y = [],[]
s = r"C:\Users\Sneh\Documents\Project\Brain Tumor Detection\Kaggle"
classification_dict = {'yes':1,'no':0}
for i in path.keys():
    for j in path[i]:
        img = cv2.imread(s + '\\' + i + '\\' + j)
        img = cv2.resize(img, (128, 128))
#         gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        x.append(img)
        y.append(classification_dict[i])  
x = np.array(x)
y = np.array(y)

In [30]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2 ,random_state=0)

In [31]:
from tensorflow.keras.utils import normalize
x_train = normalize( x_train, axis =1)
x_test = normalize( x_test, axis =1)

In [32]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

# Building a simple CNN model
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape = (128, 128, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3),  kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
#model.add(Dense(2))
model.add(Activation('sigmoid'))
# model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 activation_5 (Activation)   (None, 126, 126, 32)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 activation_6 (Activation)   (None, 61, 61, 64)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                  

In [33]:
tf.keras.utils.plot_model(model,
                          to_file="model.png",
                          show_shapes=True,
                          expand_nested=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [34]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [41]:
model.fit(x_train, y_train, 
batch_size = 16, 
verbose = 1, epochs = 15, 
validation_data = (x_test[:300], y_test[:300]),
shuffle = 'False')

Epoch 1/15
150/150 [==============================] - 50s 322ms/step - loss: 0.0141 - accuracy: 0.9946 - val_loss: 0.2453 - val_accuracy: 0.9567
Epoch 2/15
150/150 [==============================] - 47s 314ms/step - loss: 0.0133 - accuracy: 0.9937 - val_loss: 0.2845 - val_accuracy: 0.9633
Epoch 3/15
150/150 [==============================] - 48s 318ms/step - loss: 0.0365 - accuracy: 0.9858 - val_loss: 0.1922 - val_accuracy: 0.9700
Epoch 4/15
150/150 [==============================] - 50s 335ms/step - loss: 0.0059 - accuracy: 0.9983 - val_loss: 0.1772 - val_accuracy: 0.9667
Epoch 5/15
150/150 [==============================] - 49s 328ms/step - loss: 0.0048 - accuracy: 0.9983 - val_loss: 0.2183 - val_accuracy: 0.9533
Epoch 6/15
150/150 [==============================] - 49s 327ms/step - loss: 0.0044 - accuracy: 0.9983 - val_loss: 0.3154 - val_accuracy: 0.9600
Epoch 7/15
150/150 [==============================] - 49s 327ms/step - loss: 0.0018 - accuracy: 0.9992 - val_loss: 0.2328 - val_ac

In [42]:
model.save('Brain Tumor detection2.h5')

In [41]:
y_pred_class = []
for i in y_pred:
    if i > 0.25:
        y_pred_class.append(1)
    else:
        y_pred_class.append(0)

In [47]:
train_acc = model.evaluate(x_train,y_train, verbose=0)
test_acc = model.evaluate(x_test, y_test, verbose=0)

In [48]:
print('Training Acc:',train_acc)
print('Test Acc:',test_acc)

Training Acc: [3.3914242521859705e-05, 1.0]
Test Acc: [0.15384836494922638, 0.9816666841506958]


In [46]:
from keras.models import load_model
model = load_model('Brain Tumor detection.h5')